# Gallery Example: M/M/1 Queue

This is the classic M/M/1 queueing model:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS (First-Come-First-Served)

This is one of the most fundamental queueing models used for validation and benchmarking.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun galleryMM1(): Network {
    val model = Network("M/M/1")
    
    // Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
    
    // Block 2: classes
    val oclass = OpenClass(model, "myClass")
    source.setArrival(oclass, Exp(1.0))  // λ = 1
    queue.setService(oclass, Exp(2.0))   // μ = 2, so ρ = λ/μ = 0.5
    
    // Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model.link(P)
    
    return model
}

// Create the model
val model = galleryMM1()

## Theoretical Results for M/M/1

For M/M/1 with λ=1, μ=2 (ρ=0.5):
- **Utilization**: ρ = 0.5
- **Mean Queue Length**: L = ρ/(1-ρ) = 0.5/(1-0.5) = 1.0
- **Mean Response Time**: W = 1/(μ-λ) = 1/(2-1) = 1.0 seconds
- **Throughput**: X = λ = 1.0

In [ ]:
// Solve with multiple solvers - aligned with MATLAB gallery patterns

// MVA Solver (exact for open models)
val solverMVA = MVA(model)
val avgTableMVA = solverMVA.avgTable
println("MVA Solver:")
avgTableMVA.print()

// CTMC Solver
val solverCTMC = CTMC(model, "cutoff", 10)
val avgTableCTMC = solverCTMC.avgTable
println("\nCTMC Solver:")
avgTableCTMC.print()

// Fluid Solver
val solverFluid = FLD(model)
val avgTableFluid = solverFluid.avgTable
println("\nFluid Solver:")
avgTableFluid.print()